# Description

Notebook to compute metrics.

# Setup

In [5]:
%matplotlib notebook
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Imports

In [6]:
# os related
from pathlib import Path
import os 

# data related
import pandas as pd
import missingno as msno
import datetime as dt
import numpy as np


# visual related
import matplotlib.pyplot as plt
# time related

In [7]:

from fonctions import *


## Paths

In [8]:
# MUST BE IN CODE FOLDER TO WORK !
PROJECT_PATH = Path(os.getcwd()).parent

RAW_DATA = PROJECT_PATH / "RAW_DATA" 
PROC_DATA = PROJECT_PATH / "PROC_DATA"

# RAW FILES
CSV_FILE = RAW_DATA / "Individual_stock_data_csv.csv"
XLSX_FILE = RAW_DATA / "Individual_stock_data.xlsx"

# PROC FILES
PROC_STOCK = PROC_DATA / "proc_stock.csv"
PROC_CARBON = PROC_DATA / "proc_carbon.csv"

STOCK_2000 = PROC_DATA / "stock_2000.csv"

STOCK_WORK = PROC_DATA / "stock_work.csv"
STOCK2USE = PROC_DATA / "stock2use.csv"
DIC2USE = PROC_DATA / "dic2use.json"


OUT_RENDEMENT = PROC_DATA / "rendement_v1.csv"

OUT_DF_CSV = PROC_DATA / "optimize_assets.csv"

## Utils

In [5]:

from fonctions import *

# Notebook

In [9]:
return_df = pd.read_csv(OUT_DF_CSV)
return_df.index = read_file(STOCK2USE).index[104:-4]
return_df.drop(columns="Unnamed: 0", inplace=True)

In [11]:

sp500 = read_file(STOCK_WORK)
bm_return = equally_weighted(sp500)


In [42]:
return_sp = pd.DataFrame((sp500.mean(axis=1).pct_change())*100, columns=["mu"])
return_sp = return_sp.loc[return_df.index[0]:return_df.index[-1]]

In [16]:
(sp500.pct_change()).mean(axis=1)*100

timestamp
1999-12-14         NaN
1999-12-21    1.621024
1999-12-28    2.917997
2000-01-04   -2.224484
2000-01-11    4.114965
                ...   
2019-11-12    0.216510
2019-11-19    0.577101
2019-11-26    0.851971
2019-12-03   -1.712872
2019-12-10    1.596631
Length: 1044, dtype: float64

In [62]:
metrics = ["ret", "vol", "sr", "track_err", "yearly_track_error" "ex_ret", "enc", "encb", "ir", "di"]

In [63]:
rf = 0.5

In [64]:
dic = dict.fromkeys(return_df.columns)

for k in dic.keys():
    dic[k] = dict.fromkeys(metrics)

In [72]:
for pf in dic.keys():
        
    # Rendement annualisé
    dic[pf]["ret"] = return_df[pf].mean() * 52

    # Volatilité sur les 18 ans
    dic[pf]["vol"] = return_df[pf].std() * np.sqrt(52)
    
    # Ratio de sharp
    dic[pf]["sr"] = (dic[pf]["ret"] - rf) / dic[pf]["vol"]
    
     # Tracking error
    dic[pf]["track_err"] = []
    for i in range(len(return_df)//12):
        dic[pf]["track_err"].append((return_df[pf].iloc[i*12:(i+1)*12] - return_sp["mu"].iloc[i*12:(i+1)*12]).std())
    
    # Tracking global error
    dic[pf]["yearly_track_err"] = []
    for i in range(len(return_df)//(12*4)):
        dic[pf]["yearly_track_err"].append((return_df[pf].iloc[i*12*4:(i+1)*12*4] - return_sp["mu"].iloc[i*12*4:(i+1)*12*4]).std())
    
    # Excess return 
    # dic[pf]["ex_ret"] = 
    
    # Calcul de l'ENC
    # dic[pf]["enc"] = 
    
    # Information ratio
    # dic[pf]["ir"]
    
    # ENCB
    # dic[pf]["encb"]
    
    # Diversification index
    # dic[pf]["di"]